In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
from pandas.io.formats.style import Styler
from pybatfish.client.session import Session
from pybatfish.datamodel import *
from pybatfish.datamodel.answer import *
from pybatfish.datamodel.flow import *
from pybatfish.util import get_html



pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option("display.html.use_mathjax", False)

# UUID for CSS styles used by pandas styler.
# Keeps our notebook HTML deterministic when displaying dataframes
_STYLE_UUID = "pybfstyle"

bf = Session(host="localhost")


class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)


def show(df):
    """
    Displays a dataframe as HTML table.

    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """
    if isinstance(df, TableAnswer):
        df = df.frame()

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    display(
        MyStyler(df)
        .set_uuid(_STYLE_UUID)
        .format(get_html)
        .set_properties(**{"text-align": "left", "vertical-align": "top"})
    )

In [11]:

NETWORK_NAME = 'example'
SNAPSHOT_NAME = 'example_snapshot'

SNAPSHOT_PATH = 'example_snapshot'

bf.set_network(NETWORK_NAME)
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)
    
    

'example_snapshot'

In [14]:
bf.q.initIssues().answer().frame()

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,None,[configs/R2.cfg:[32]],Parse warning,This feature is not currently supported,as-set,[agg_as_set aggregate_address_rb_stanza router_bgp_stanza_tail router_bgp_stanza stanza cisco_configuration]


In [12]:
bf.q.bgpSessionStatus().answer().frame()

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Established_Status
0,r1,default,400,None,3.0.0.3,512,r2,None,3.0.0.2,['IPV4_UNICAST'],EBGP_SINGLEHOP,ESTABLISHED
1,r1,default,400,None,100.0.0.1,400,None,None,100.0.0.2,[],IBGP,NOT_COMPATIBLE
2,r1,default,400,None,100.1.0.1,400,None,None,100.1.0.2,[],IBGP,NOT_COMPATIBLE
3,r2,default,512,None,4.0.0.2,768,r3,None,4.0.0.3,['IPV4_UNICAST'],EBGP_SINGLEHOP,ESTABLISHED
4,r2,default,512,None,3.0.0.2,400,r1,None,3.0.0.3,['IPV4_UNICAST'],EBGP_SINGLEHOP,ESTABLISHED
5,r3,default,768,None,4.0.0.3,512,r2,None,4.0.0.2,['IPV4_UNICAST'],EBGP_SINGLEHOP,ESTABLISHED


In [13]:
df1 = bf.q.bgpRib(nodes='R3').answer().frame()
show(df1)
# df2 = bf.q.bgpRib(nodes='R2').answer().frame()
# df3 = bf.q.bgpRib(nodes='R3').answer().frame()
# # df4 = bf.q.bgpRib(nodes='R4').answer().frame()

# newdf = (pd.concat([df1, df2, df3])).reset_index(drop=True)

# newdf = newdf[newdf['Network'] == '100.0.10.0/24'][["Node", "AS_Path"]].reset_index(drop=True)

,Node,VRF,Network,Status,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,AS_Path,Metric,Local_Pref,Communities,Origin_Protocol,Origin_Type,Originator_Id,Received_From_IP,Received_Path_Id,Cluster_List,Tunnel_Encapsulation_Attribute,Weight,Tag
0,r3,default,100.0.0.0/8,BEST,ip 4.0.0.2,4.0.0.2,dynamic,bgp,512,0,50,,bgp,igp,4.0.0.2,4.0.0.2,None,None,None,0,None


In [42]:
import csv
import json
import glob

files = glob.glob('../gobgp/results/*.txt')

comparison_file = '../comparison.csv'
comparison_dict = {}

with open('../../all_test_cases.json', mode='r') as f:
    test_cases = json.load(f)
    
with open(comparison_file, mode='w') as csvfile:
    fieldnames = ['Test Case', 'Batfish R2 AS Path', 'GoBGP R2 AS Path', 'Batfish R3 AS Path', 'GoBGP R3 AS Path', 'Match']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    data = []
    for i in range(len(files)):
        with open('results/test_case' + str(i) + '.txt', 'r') as f:
            batfish_test_result = f.read().split('\n')
            
        # print(batfish_test_result)
            
        with open('../gobgp/results/' + str(i) + '.txt', 'r') as f:
            gobgp_result = f.read().split('\n')
        
        batfish_test_result = batfish_test_result[2:]
        gobgp_test_result = [gobgp_result[0], gobgp_result[2]]
        
        batfish_as_paths = []
        for line in batfish_test_result:
            tokens = line.split()
            as_path = ' '.join(tokens[2:-1])
            batfish_as_paths.append(as_path)
        
        gobgp_as_paths = []
        for line in gobgp_test_result:
            tokens = line.split(':')
            as_path = tokens[1].strip()
            if as_path != '':
                gobgp_as_paths.append(as_path)
                
        # print(batfish_as_paths)
        # print(gobgp_as_paths)

        #print(test_cases[i])
        result_row = {
            'Test Case': f"Origin AS: {test_cases[i][0]}\nRouter 2: {test_cases[i][2]}\nRouter 3: {test_cases[i][3]}\nRemove Private AS: {test_cases[i][4]}\nReplace AS: {test_cases[i][5]}",
            'Batfish R2 AS Path': batfish_as_paths[0],
            'GoBGP R2 AS Path': gobgp_as_paths[0],
            'Batfish R3 AS Path': batfish_as_paths[1] if len(batfish_as_paths) > 1 else 'None',
            'GoBGP R3 AS Path': gobgp_as_paths[1] if len(gobgp_as_paths) > 1 else 'None',
        }
        
        if batfish_as_paths == gobgp_as_paths:
            result_row['Match'] = 'Yes'
        else:
            result_row['Match'] = 'No'
            
        data.append(result_row)
    
    writer.writerows(data)
        
    